In [ ]:
import copy
import optuna
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from bcmix import *

In [ ]:
fed = pd.read_csv("realdata/FEDFUNDS.csv")
pce = pd.read_csv("realdata/EXPINF1YR.csv")

In [ ]:
X = fed["FEDFUNDS"].values
y = pce["EXPINF1YR"].values

In [ ]:
CP = 79

X0 = X[:CP].reshape((-1, 1))
y0 = y[:CP]
model0 = LinearRegression().fit(X0, y0)
y0_hat = model0.predict(X0)
res0 = y0 - y0_hat

X1 = X[CP:].reshape((-1, 1))
y1 = y[CP:]
model1 = LinearRegression().fit(X1, y1)
y1_hat = model1.predict(X1)
res1 = y1 - y1_hat

In [ ]:
fed.iloc[CP]

In [ ]:
SIGMA = ((res0.T @ res0 + res1.T @ res1) / len(y)).item() ** 0.5
ALPHA0 = 1.42 / SIGMA
ALPHA1 = 0.25 / SIGMA
BETA0 = 0.2 / SIGMA
BETA1 = -0.1 / SIGMA

In [ ]:
N_TRIALS = 25
DATA_LEN = len(y)
P = 0.025

In [ ]:
mean_true = np.array([[2.0 / SIGMA], [0.0]])
covm_true = np.array([[1.0, 0.0], [0.0, 1.0]])
# prior
canonical_0 = np.array([[2.0 / SIGMA], [0.0]])
precision_0 = np.array([[1.0, 0.0], [0.0, 1.0]])
logcon_0 = (np.linalg.slogdet(precision_0)[1] - (canonical_0.T @ np.linalg.inv(precision_0) @ canonical_0).item()) / 2
states = {0: {"can": canonical_0, "pre": precision_0, "log": logcon_0, "pit": 0.0}}

In [ ]:
def objective_bcmix(trial):
    a = trial.suggest_float('a', ACTION_RANGE[0], ACTION_RANGE[1])
    q = q_myopic_with_change(states_i, a, alpha_i, beta_i, mean_true, covm_true, p=P)
    return q

def objective(trial):
    a = trial.suggest_float('a', ACTION_RANGE[0], ACTION_RANGE[1])
    q = q_myopic_without_change(canonical_i, precision_i, a, alpha_i, beta_i)
    return q

In [ ]:
for i in range(1):
    # initialize
    simresult_bcmix_i = np.full((DATA_LEN, 6 * (1 + M1 + M2) + 4), 0.0)
    simresult_i = np.full((DATA_LEN, 9), np.nan)
    canonical_i, precision_i, states_i = canonical_0, precision_0, copy.deepcopy(states)
    for j in range(DATA_LEN):
        alpha_i = ALPHA0 if j < CP else ALPHA1
        beta_i = BETA0 if j < CP else BETA1
        # current state
        # bcmix model
        simresult_bcmix_i[j, 0] = alpha_i
        simresult_bcmix_i[j, 1] = beta_i
        for m, s in states_i.items():
            covm_bcmix_i = np.linalg.inv(s["pre"])
            mean_bcmix_i = covm_bcmix_i @ s["can"]
            simresult_bcmix_i[j, m * 6 + 2] = mean_bcmix_i[0][0]
            simresult_bcmix_i[j, m * 6 + 3] = mean_bcmix_i[1][0]
            simresult_bcmix_i[j, m * 6 + 4] = covm_bcmix_i[0][0]
            simresult_bcmix_i[j, m * 6 + 5] = covm_bcmix_i[0][1]
            simresult_bcmix_i[j, m * 6 + 6] = covm_bcmix_i[1][1]
            simresult_bcmix_i[j, m * 6 + 7] = s["pit"]
        # old model
        simresult_i[j, 0] = alpha_i
        simresult_i[j, 1] = beta_i
        covm_i = np.linalg.inv(precision_i)
        mean_i = covm_i @ canonical_i
        simresult_i[j, 2] = mean_i[0][0]
        simresult_i[j, 3] = mean_i[1][0]
        simresult_i[j, 4] = covm_i[0][0]
        simresult_i[j, 5] = covm_i[0][1]
        simresult_i[j, 6] = covm_i[1][1]
        # select action
        # bcmix model
        x_myopic_bcmix = myopic_mix(states_i, p=P)
        ACTION_RANGE = (x_myopic_bcmix - 2.5, x_myopic_bcmix + 2.5)
        study_bcmix = optuna.create_study(direction="maximize")
        study_bcmix.optimize(objective_bcmix, n_trials=N_TRIALS)
        a_bcmix = study_bcmix.best_trial.params['a']
        simresult_bcmix_i[j, -2] = a_bcmix
        # old model
        x_myopic = myopic(canonical_i, precision_i)
        ACTION_RANGE = (x_myopic - 2.5, x_myopic + 2.5)
        study = optuna.create_study(direction="maximize")
        study.optimize(objective, n_trials=N_TRIALS)
        a = study.best_trial.params['a']
        simresult_i[j, 7] = a
        # update state
        # bcmix model
        y_bcmix = env_response(a_bcmix, alpha_i, beta_i, mean_true=None, covm_true=None, p=0)[0]
        simresult_bcmix_i[j, -1] = y_bcmix * GAMMA
        states_i = update_with_change(states_i, a_bcmix, y_bcmix, p=P)
        # old model
        y = env_response(a, alpha_i, beta_i)[0]
        simresult_i[j, 8] = y * GAMMA
        canonical_i, precision_i = update_without_change(canonical_i, precision_i, a, y)
    np.save("realdata\inf_cgbcmix_" + str(i) + ".npy", simresult_bcmix_i)
    np.save("realdata\inf_cgold_" + str(i) + ".npy", simresult_i)

In [ ]:
simresult_bcmix = np.zeros(DATA_LEN)
simresult = np.zeros(DATA_LEN)

sim = 10
for i in range(sim):
    simresult_bcmix += np.load("realdata\inf_cgbcmix_" + str(i) + ".npy")[:, -1]
    simresult += np.load("realdata\inf_cgold_" + str(i) + ".npy")[:, -1]

simresult_bcmix /= sim
simresult /= sim
plt.plot(simresult_bcmix, color="red")
plt.plot(simresult, color="blue")
plt.plot(pce["EXPINF1YR"].values, color="black")

In [ ]:
simresult_bcmix = np.zeros(DATA_LEN)
simresult = np.zeros(DATA_LEN)

sim = 10
for i in range(sim):
    simresult_bcmix += np.load("realdata\inf_cgbcmix_" + str(i) + ".npy")[:, -2]
    simresult += np.load("realdata\inf_cgold_" + str(i) + ".npy")[:, -2]

simresult_bcmix /= sim
simresult /= sim
plt.plot(simresult_bcmix, color="red")
plt.plot(simresult, color="blue")
plt.plot(pce["FEDFUNDS"].values, color="black")